<a href="https://colab.research.google.com/github/lncknight/Countable/blob/master/pdf_table_sc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt update -y
!apt install ghostscript python3-tk

!pip install camelot-py tabula-py
!pip install ghostscript

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:13 https://developer.

In [ ]:
import camelot
import pandas as pd

pd.set_option('display.max_columns', None)

# PDF file to extract tables from
file = "2021-11.pdf"

def parse_pdf():
  # ref: https://stackoverflow.com/questions/53209335/python-camelot-borderless-table-extraction-issue
  # ref: https://camelot-py.readthedocs.io/en/master/user/advanced.html#contour
  # tables = camelot.read_pdf(file, pages="1", flavor='stream', edge_tol=500)
  tables = camelot.read_pdf(file, pages="1,2", kind='contour', flavor='stream', edge_tol=500)
  # tables = camelot.read_pdf(file, pages="1", flavor='lattice')

  # number of tables extracted
  print("Total tables extracted:", tables.n)
  # print(type(tables))

  i = 0
  # print(tables[0])
  # print(tables[1])
  dfs = list()
  for table in tables:
    # print(i)
    
    df = table.df
    df.to_csv("raw-{}.csv".format(str(i)))
    # print(df[1])
    
    i += 1

    dfs.append(df)
    # return df

  return dfs

def get_head_name_map(head):
  head_name_map = {
    '0': 'item'
  }
  j = 0
  for v in head:
    j += 1
    v = str(v)
    # print(v, type(v))
    if 'HKD Amount' in v:
      # print('------1', v, j)
      head_name_map[str(j-2)] = 'amount'
    if 'Transaction Reference' in v:
      head_name_map[str(j-2)] = 'ref'
      # print('------2', v, j)
  # print(head_name_map)

  return head_name_map

def parse_csv():
  dfs = list()
  try:
    index = 0
    while True:
      i = index # copy index
      index += 1

      raw_file = 'raw-{}.csv'.format(i)
      print('[{}] process raw file {}'.format(i, raw_file))

      # restore df
      df = pd.read_csv(raw_file)

      # determine boundaries
      start_index = df.loc[df['2'] == 'Transaction Reference'].index
      # re-try
      if (len(start_index) == 0):
        print('[{}] re-trying another column'.format(i))
        start_index = df.loc[df['1'] == 'Transaction Reference'].index

      end_index = df.loc[df['0'].str.contains('To be continued') == True].index

      if (len(start_index) == 0):
        print('[{}] not found head row, SKIP'.format(i))
        # print(df)
        continue
      
      # head row
      head = df.iloc[start_index[0]]

      # filter boundaries
      if (len(end_index) == 0):
        print('[{}] no end row detected, scan till the end'.format(i))
        df = df.loc[(start_index[0]+1):]
      else:
        df = df.loc[(start_index[0]+1):(end_index[0]-1)]

      # column map
      head_name_map = get_head_name_map(head)
      df.rename(columns=head_name_map, inplace=True)

      # pick columns
      df = df[['item', 'ref', 'amount']]

      dfs.append(df)  

  except FileNotFoundError:
    print('[{}] file not file, SKIP'.format(i))
    return dfs  


In [ ]:
dfs = parse_csv()

[0] process raw file raw-0.csv
[0] no end row detected, scan till the end
[1] process raw file raw-1.csv
[1] re-trying another column
[1] not found head row, SKIP
[2] process raw file raw-2.csv
[2] file not file, SKIP


In [ ]:
dfs = parse_pdf()

Total tables extracted: 2


In [ ]:
# dfs[0]
df = pd.concat(dfs, ignore_index=True)
ddf = df

# df.rename(columns={
#     0: 'row',
#     '0': 'item',
#     '1': '_del',
#     '2': 'ref',
#     '3': '_del2',
#     '4': '_del3',
#     '5': 'amount',
# }, inplace=True)

# start_index = df.loc[df['ref'] == 'Transaction Reference'].index

# ddf = df.loc[13+1:]
ddf = ddf[ddf['ref'].str.len() > 6]
# ddf = ddf[~ddf['ref'].isnull()]
ddf = ddf[~ddf['item'].str.contains('THANK YOU')]
dddf = ddf
# del dddf['_del']

dddf.to_csv('result.csv')



from re import sub
from decimal import Decimal

# i = 17
# while True:
#   try:
#     i += 1
#     print(i, type(dddf['amount'][i]))
#     print((dddf['amount'][i]))
#     Decimal(sub(r'[^\d.]', '', dddf['amount'][i]))
#   except KeyError:
#     continue

dddf['amount'] = dddf['amount'].apply(lambda x: Decimal(sub(r'[^\d.]', '', (x))))
# money = '$6,150,593.22'
# value = 
# value
# pd.to_numeric(dddf['amount'])
# df.apply(lambda x: pd.Series([1, 2], index=['foo', 'bar']), axis=1)

print('total amount:', dddf.amount.sum())

dddf


total amount: 4097.00


,item,ref,amount
1,22 Oct\nPAYME FROM HSBC HK HK,74896021295092900281485,23.00
2,PAYME FROM HSBC HK HK,74896021295092900286203,30.00
3,23 Oct\nGENKI SUSHI − 5992 TSEUNG KWAN HK,74364761298859920110030,326.70
4,24 Oct\nKFC − SHOP 193 TSEUNG KWAN HK,74364761300481930230124,60.00
5,FRESH − LOHAS 481HONG KONG HK,74552351298481000416922,223.00
6,Goobne Sai Kung HK,74263761297007412520554,529.00
7,PARKNSHOP.COM HONG KONG HK,74518341297502000005608,659.10
8,25 Oct\nCIRCLE K 591 079HONG KONG HK,74552351300079000086376,180.00
9,27 Oct\nCSL PREPAID SIM CARDKOWLOON HK,74896021300092900039241,50.00
10,PAYME FROM HSBC HK HK,74896021300092900376205,55.00


In [ ]:
# testing
i = 1
j = i
i += 1
print(j)


from re import sub
from decimal import Decimal

money = '$6,150,593.22'
value = Decimal(sub(r'[^\d.]', '', money))
value

1


'6150593.22'